In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', 100)
import sys
sys.path.append('../../..')
sys.path.append('../')
sys.path.append('../../')

from config import *
from src.plot_utils import _prettify_ax, add_panel_text
from src.vax_tables import get_national_vax_df

In [ ]:
SZ=16
title_fontsize=16
every_x_axis_ticks=7

vax_total_pop_file = True

color_firstvax = 'cornflowerblue'
color_secondvax = 'limegreen'
color_recovered = 'silver'

last_date = pd.to_datetime('2021-02-24')

# Cumulative Doses

In [ ]:
national_regroup_dict = {'0-19': '16+', '20-29': '16+', '30-39': '16+', '40-49': '16+', 
                         '50-59': '16+', '60-69': '16+', '70-79': '16+', '80-89': '16+', '90+': '16+'}

national_vax_df = get_national_vax_df(minimal_age=16, regroup_dict=national_regroup_dict,
                                      vax_total_pop_file=vax_total_pop_file)
national_vax_df = national_vax_df.reset_index().set_index('date')
national_vax_df = national_vax_df.loc[:last_date]
national_vax_df.head()

In [ ]:
# fig, axes = plt.subplots(2,2, figsize=(14, 10))
fig, axes = plt.subplots(1,2, figsize=(16, 6))


# ax = axes[0, 0]
ax = axes[0]
add_panel_text(ax,'a')

tmp = national_vax_df.loc[national_vax_df.age_group == '16+', ['first_dose', 'second_dose']]
ax.bar(tmp.index, tmp.sum(axis=1).values, color=color_secondvax, label='Second dose administered')
ax.bar(tmp.index, tmp['first_dose'].values, color=color_firstvax, label='First dose administered')
ax.set_title('National daily doses administered', fontsize=title_fontsize)
ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Number of doses', fontsize=16)
ax.set_ylim([0,300000])
_prettify_ax(ax, tmp.index, loc='upper left')
ax.tick_params(axis='both', which='major', labelsize=14)


# ax = axes[0, 1]
ax = axes[1]
add_panel_text(ax,'b')

tmp = national_vax_df.loc[national_vax_df.age_group == '16+', 'first_dose_perc']
ax.stackplot(tmp.index, tmp.values, color=color_firstvax, labels=['Vaccinated 1st dose'])

tmp = national_vax_df.loc[national_vax_df.age_group == '16+', 'second_dose_perc']
ax.stackplot(tmp.index, tmp.values, color=color_secondvax, labels=['Vaccinated 2nd dose'])

# tmp = national_vax_df.loc[national_vax_df.age_group == '16+', 'recovered_perc']
# ax.stackplot(tmp.index, tmp.values, color=color_recovered, labels=['Recovered'])


ax.set_title(f'National', fontsize=title_fontsize)
ax.set_ylabel('Vaccinated\nfrom eligible population [%]', fontsize=16)
ax.set_xlabel('Date', fontsize=16)

ax.set_ylim([0, 119])
ax.tick_params(axis='both', which='major', labelsize=14)

ax.axvline(x=pd.to_datetime('2020-12-20'), ls='--', color='k', lw=2)#, label='60+ vaccination')
ax.text(x=pd.to_datetime('2020-12-21'), y=58, s='60+ and\nhealthcare\nworkers', fontsize=14)

ax.axvline(x=pd.to_datetime('2021-01-12'), ls='--', color='r', lw=2)#, label='55+ vaccination')
ax.text(x=pd.to_datetime('2021-01-07'), y=63, s='55+', fontsize=14, color='r')

ax.axvline(x=pd.to_datetime('2021-01-19'), ls='--', color='g', lw=2)#, label='40+ vaccination')
ax.text(x=pd.to_datetime('2021-01-14'), y=63, s='40+', fontsize=14, color='g')

ax.axvline(x=pd.to_datetime('2021-01-21'), ls='--', color='b', lw=2)#, label='40+ vaccination')
ax.text(x=pd.to_datetime('2021-01-22'), y=83, s='40+\nand\n16-18', fontsize=14, color='b')

ax.axvline(x=pd.to_datetime('2021-01-28'), ls='--', color='purple', lw=2)#, label='40+ vaccination')
ax.text(x=pd.to_datetime('2021-01-29'), y=83, s='35+\nand\n16-18', fontsize=14, color='purple')

ax.axvline(x=pd.to_datetime('2021-02-04'), ls='--', color='gray', lw=2)#, label='40+ vaccination')
ax.text(x=pd.to_datetime('2021-02-05'), y=90, s='16+', fontsize=14, color='gray')

_prettify_ax(ax, tmp.index, loc='upper left')


fig.tight_layout()

# Doses per age group

In [ ]:
national_regroup_dict = {'0-19': '0-19', '20-29': '20-29', '30-39': '30-39', '40-49': '40-49', 
                         '50-59': '50-59', '60-69': '60-69', '70-79': '70-79', '80-89': '80+', '90+': '80+'}

national_vax_df = get_national_vax_df(minimal_age=0, regroup_dict=national_regroup_dict,
                                      vax_total_pop_file=vax_total_pop_file)
national_vax_df = national_vax_df.loc[:last_date]
national_vax_df = national_vax_df.reset_index()

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(12, 20))

for idx, ag in enumerate(national_vax_df.age_group.unique()):
    ax = axes[idx // 2, idx % 2]
    tmp = national_vax_df[national_vax_df['age_group'] == ag].groupby(
        'date')[['first_dose', 'second_dose']].sum()
    
    ax.bar(tmp.index, tmp.sum(axis=1).values, color=color_secondvax, label='Second dose administered')
    ax.bar(tmp.index, tmp['first_dose'].values, color=color_firstvax, label='First dose administered')
    ax.set_title(f'Daily doses for age group {ag}', fontsize=title_fontsize)
    ax.set_ylabel('Number of Doses', fontsize=16)
    ax.set_xlabel('Date', fontsize=16)
    ax.set_ylim([0, 60000])
    _prettify_ax(ax, tmp.index, loc='upper left')
        
fig.tight_layout()

# Vaccinated per age group

In [ ]:
national_regroup_dict = {'0-19': '0-19', '20-29': '20-29', '30-39': '30-39', '40-49': '40-49', 
                         '50-59': '50-59', '60-69': '60-69', '70-79': '70-79', '80-89': '80+', '90+': '80+'}

national_vax_df = get_national_vax_df(minimal_age=0, regroup_dict=national_regroup_dict,
                                      vax_total_pop_file=vax_total_pop_file)
national_vax_df = national_vax_df.reset_index().set_index('date')
national_vax_df = national_vax_df.loc[:last_date]
national_vax_df.head()

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(20, 28))

for idx, ag in enumerate(national_vax_df.age_group.unique()):
    ax = axes[idx // 2, idx % 2]
    
    tmp = national_vax_df.loc[national_vax_df.age_group == ag, 'first_dose_perc']
    ax.stackplot(tmp.index, tmp.values, color=color_firstvax, labels=['Vaccinated 1st dose'])
    
    tmp = national_vax_df.loc[national_vax_df.age_group == ag, 'second_dose_perc']
    ax.stackplot(tmp.index, tmp.values, color=color_secondvax, labels=['Vaccinated 2nd dose'])

    ax.set_title(f'Age group {ag}', fontsize=24)
    ax.set_ylabel('Vaccinated or recovered [%]', fontsize=22)
    ax.set_xlabel('Date', fontsize=22)
    ax.set_ylim([0, 100])
    _prettify_ax(ax, tmp.index, loc='upper left')
        
    ax.tick_params(axis='both', which='major', labelsize=16)

fig.tight_layout()